# Notebook 3 – First baseline model

Purpose:
- Load truth data from Hopsworks feature store.
- Define feature set and train the first baseline model.
- Run predictions on a holdout set and evaluate (e.g., MAE/MSE).
- Log model version and metrics (Hopsworks/MLflow/file).
- Persist the trained model so Notebook 4 can load it.

Notes:
- Document chosen model type (e.g., XGBoost, RandomForest, LSTM) and tested hyperparameters.
- Record performance and next experiments to try.
